In [57]:
import pandas as pd
from backend.data_center.kline_data.kline_data_collector import *
tv = KlineDataCollector()
file_abspath = tv.get_abspath(symbol='BTC', interval=Interval.in_daily)
df = pd.read_csv(f"{file_abspath}")
df

,datetime,symbol,open,high,low,close,volume
0,2017-08-17 08:00:00,Binance:BTCUSDT,4261.48,4485.39,4200.74,4285.08,795.150377
1,2017-08-18 08:00:00,Binance:BTCUSDT,4285.08,4371.52,3938.77,4108.37,1199.888264
2,2017-08-19 08:00:00,Binance:BTCUSDT,4108.37,4184.69,3850.00,4139.98,381.309763
3,2017-08-20 08:00:00,Binance:BTCUSDT,4120.98,4211.08,4032.62,4086.29,467.083022
4,2017-08-21 08:00:00,Binance:BTCUSDT,4069.13,4119.62,3911.79,4016.00,691.743060
...,...,...,...,...,...,...,...
2577,2024-09-06 08:00:00,Binance:BTCUSDT,56180.00,57008.00,52550.00,53962.97,54778.901400
2578,2024-09-07 08:00:00,Binance:BTCUSDT,53962.97,54850.00,53745.54,54160.86,16765.751580
2579,2024-09-08 08:00:00,Binance:BTCUSDT,54160.86,55318.00,53629.01,54869.95,16285.976920
2580,2024-09-09 08:00:00,Binance:BTCUSDT,54869.95,58088.00,54591.96,57042.00,32595.282280


In [59]:
df['prev_low'] = df['low'].shift(1)
df['prev_high'] = df['high'].shift(1)
df.dropna(inplace=True)
# calculate the close price in a quarter at the end of the day
df['attack_buy_pattern'] = df['close'] < df['prev_low'] 
df['prev_is_abp'] = df['attack_buy_pattern'].shift(1)
df.dropna(inplace=True)
df

,datetime,symbol,open,high,low,close,volume,prev_low,prev_high,attack_buy_pattern,prev_is_abp
4,2017-08-21 08:00:00,Binance:BTCUSDT,4069.13,4119.62,3911.79,4016.00,691.743060,4032.62,4211.08,True,False
5,2017-08-22 08:00:00,Binance:BTCUSDT,4016.00,4104.82,3400.00,4040.00,966.684858,3911.79,4119.62,False,True
6,2017-08-23 08:00:00,Binance:BTCUSDT,4040.00,4265.80,4013.89,4114.01,1001.136565,3400.00,4104.82,False,False
7,2017-08-24 08:00:00,Binance:BTCUSDT,4147.00,4371.68,4085.01,4316.01,787.418753,4013.89,4265.80,False,False
8,2017-08-25 08:00:00,Binance:BTCUSDT,4316.01,4453.91,4247.48,4280.68,573.612740,4085.01,4371.68,False,False
...,...,...,...,...,...,...,...,...,...,...,...
2577,2024-09-06 08:00:00,Binance:BTCUSDT,56180.00,57008.00,52550.00,53962.97,54778.901400,55643.65,58327.07,True,False
2578,2024-09-07 08:00:00,Binance:BTCUSDT,53962.97,54850.00,53745.54,54160.86,16765.751580,52550.00,57008.00,False,True
2579,2024-09-08 08:00:00,Binance:BTCUSDT,54160.86,55318.00,53629.01,54869.95,16285.976920,53745.54,54850.00,False,False
2580,2024-09-09 08:00:00,Binance:BTCUSDT,54869.95,58088.00,54591.96,57042.00,32595.282280,53629.01,55318.00,False,False


In [62]:
df['buy_sig'] = (df['prev_is_abp'] == True) & (df['high']>df['prev_high'])
df[df['buy_sig']==True].size

1008

In [63]:
# to check
df['take_profit'] = (df['prev_is_abp'] == True) & (df['close']>df['prev_high'])
df[df['take_profit']==True].size

689

In [64]:
689/1008

0.683531746031746

In [32]:
# condition1. prev day is rising
# condition2. today is rising
# condition3. today price is at the bottom quarter
df.dropna(inplace=True)
df['attack_buy_pattern'] = (df['prev_close'] > df['prev_open']) & (df['close'] > df['open']) & ((df['close']-df['low'])/(df['high']-df['low']) < 0.25)


In [37]:
df[df['attack_buy_pattern']==True]
# df

,datetime,symbol,open,high,low,close,volume,prev_close,prev_open,attack_buy_pattern,next_close,next_open,will_increase
187,2018-02-20 08:00:00,Binance:BTCUSDT,11147.11,11786.01,11100.59,11200.99,48153.354288,11153.00,10375.01,True,10437.60,11195.07,False
430,2018-10-21 08:00:00,Binance:BTCUSDT,6585.72,6663.90,6580.00,6590.11,6515.357792,6588.40,6528.88,True,6581.20,6590.12,False
489,2018-12-19 08:00:00,Binance:BTCUSDT,3653.00,3912.73,3622.55,3662.22,94137.341014,3652.98,3509.03,True,4049.62,3660.01,True
871,2020-01-05 08:00:00,Binance:BTCUSDT,7354.19,7495.00,7318.00,7358.75,38331.085604,7354.11,7345.00,True,7758.00,7357.64,True
1311,2021-03-20 08:00:00,Binance:BTCUSDT,58030.01,59880.00,57820.17,58102.28,44476.941776,58030.01,57641.00,True,57351.56,58100.02,False
1945,2022-12-14 08:00:00,Binance:BTCUSDT,17775.82,18387.95,17660.94,17803.15,266681.222090,17774.70,17208.93,True,17356.34,17804.01,False


In [34]:
df['next_close'] = df['close'].shift(-1)
df['next_open'] = df['open'].shift(-1)
df['will_increase'] = df['next_close'] > df['next_open']
df

,datetime,symbol,open,high,low,close,volume,prev_close,prev_open,attack_buy_pattern,next_close,next_open,will_increase
1,2017-08-18 08:00:00,Binance:BTCUSDT,4285.08,4371.52,3938.77,4108.37,1199.888264,4285.08,4261.48,False,4139.98,4108.37,True
2,2017-08-19 08:00:00,Binance:BTCUSDT,4108.37,4184.69,3850.00,4139.98,381.309763,4108.37,4285.08,False,4086.29,4120.98,False
3,2017-08-20 08:00:00,Binance:BTCUSDT,4120.98,4211.08,4032.62,4086.29,467.083022,4139.98,4108.37,False,4016.00,4069.13,False
4,2017-08-21 08:00:00,Binance:BTCUSDT,4069.13,4119.62,3911.79,4016.00,691.743060,4086.29,4120.98,False,4040.00,4016.00,True
5,2017-08-22 08:00:00,Binance:BTCUSDT,4016.00,4104.82,3400.00,4040.00,966.684858,4016.00,4069.13,False,4114.01,4040.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2577,2024-09-06 08:00:00,Binance:BTCUSDT,56180.00,57008.00,52550.00,53962.97,54778.901400,56180.00,57970.90,False,54160.86,53962.97,True
2578,2024-09-07 08:00:00,Binance:BTCUSDT,53962.97,54850.00,53745.54,54160.86,16765.751580,53962.97,56180.00,False,54869.95,54160.86,True
2579,2024-09-08 08:00:00,Binance:BTCUSDT,54160.86,55318.00,53629.01,54869.95,16285.976920,54160.86,53962.97,False,57042.00,54869.95,True
2580,2024-09-09 08:00:00,Binance:BTCUSDT,54869.95,58088.00,54591.96,57042.00,32595.282280,54869.95,54160.86,False,56689.92,57042.01,False


In [35]:
avg_rate = df[df['will_increase']==True].size / df.size
avg_rate

0.5133669120495932

In [36]:
attack_buy_increase_rate = df[(df['will_increase']==True) & (df['attack_buy_pattern'] == True)].size / df[df['attack_buy_pattern']].size
attack_buy_increase_rate

0.3333333333333333

In [26]:
df[(df['will_increase']==True) & (df['attack_buy_pattern'] == True)]

,datetime,symbol,open,high,low,close,volume,prev_close,prev_open,attack_buy_pattern,next_close,next_open,will_increase
6,2017-08-23 08:00:00,Binance:BTCUSDT,4040.00,4265.80,4013.89,4114.01,1001.136565,4040.00,4016.00,True,4316.01,4147.00,True
20,2017-09-06 08:00:00,Binance:BTCUSDT,4366.49,4662.87,4335.26,4619.77,807.363726,4366.47,4106.97,True,4691.61,4619.77,True
30,2017-09-16 08:00:00,Binance:BTCUSDT,3674.01,3950.00,3470.66,3714.95,1297.563953,3700.00,3188.01,True,3699.99,3685.23,True
31,2017-09-17 08:00:00,Binance:BTCUSDT,3685.23,3748.21,3499.02,3699.99,682.171210,3714.95,3674.01,True,4035.01,3690.00,True
50,2017-10-06 08:00:00,Binance:BTCUSDT,4318.99,4417.00,4292.00,4369.00,506.529176,4292.43,4208.59,True,4423.00,4369.00,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2523,2024-07-14 08:00:00,Binance:BTCUSDT,59204.01,61420.69,59194.01,60797.91,21178.339070,59204.02,57889.09,True,64724.14,60797.91,True
2524,2024-07-15 08:00:00,Binance:BTCUSDT,60797.91,64900.00,60632.30,64724.14,38690.978200,60797.91,59204.01,True,65043.99,64724.06,True
2529,2024-07-20 08:00:00,Binance:BTCUSDT,66660.01,67598.00,66222.46,67139.96,14386.924340,66660.00,63987.92,True,68165.34,67139.97,True
2564,2024-08-24 08:00:00,Binance:BTCUSDT,64037.24,64494.50,63531.00,64157.01,15857.156160,64037.24,60375.83,True,64220.00,64157.02,True
